# CNN for Sentence Classification 논문 구현

#### Git - Commit Message Convention

* 제가 대략적으로 framework을 잡아놓은 것이니, 담당하시는 부분에 수정이 필요하시면 마크다운 양식만 유지한 채 수정해주시면 됩니다!
* 작은 단위의 작업이 끝날 때 마다 git add, commit, push 해주시면 됩니다! (push하고 슬랙에 공유 부탁드려요! 화이팅!)

* git commit message는 다음의 양식을 참고해주세요!
    * 처음으로 코드 완료했을 때 git commit -m "동사 명사"
    ```ex) git commit -m "Fill and replace NaN values"```
    
    * commit 했던 코드를 수정했을 때 git commit -m "Update 수정한 내용"
    ```ex) git commit -m "Update Word embedding"```


# 1. 데이터 load 및 EDA

## 1) 네이버 영화 리뷰 데이터 불러오기

In [3]:
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
train = pd.read_csv("ratings_train.txt", sep='\t')
test = pd.read_csv("ratings_test.txt", sep='\t')

In [3]:
train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [4]:
train.info()    #document에 null값 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        150000 non-null  int64 
 1   document  149995 non-null  object
 2   label     150000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [5]:
test.info()    #document에 null값 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        50000 non-null  int64 
 1   document  49997 non-null  object
 2   label     50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [5]:
train[train['document'].isnull()]     #갯수가 별로 없고 대체할 방법이 없으므로 결측치 제거

,id,document,label
25857,2172111,NaN,1
55737,6369843,NaN,1
110014,1034280,NaN,0
126782,5942978,NaN,0
140721,1034283,NaN,0


In [6]:
## 결측치 제거
train.dropna(inplace=True)
test.dropna(inplace=True)

## 2) 리뷰 데이터 EDA

## 3) Sentences, labels 생성

In [31]:
import numpy as np

In [32]:
# sentence는 tokenize하고 생성
# label만 numpy로 생성
train_labels = np.array(train.label)
test_labels = np.array(test.label)

str

# 2. Data Preprocessing

## 1) Tokenizer

* 어절 단위
* 형태소 단위
* Subword 단위 중 선택

In [34]:
# 영어, 한글만 포함하고 나머지 제거
import re

def preprocess(text):
  text = re.sub(r"[^A-Za-zㄱ-ㅎㅏ-ㅣ가-힣 ]","", text) 
  return text

train['document'] = train.document.apply(lambda x : preprocess(x))
test['document'] = test.document.apply(lambda x : preprocess(x))

train.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [36]:
# Mecab으로 형태소 분석, 불용어 제거
from konlpy.tag import Mecab

tokenizer = Mecab()
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

train_tokens = []
test_tokens = []

for sentence in train['document']:
    tokenized_sentence = tokenizer.morphs(sentence) 
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] 
    train_tokens.append(stopwords_removed_sentence)

for sentence in test['document']:
    tokenized_sentence = tokenizer.morphs(sentence) 
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] 
    test_tokens.append(stopwords_removed_sentence)

In [39]:
train_tokens[:10]

[['아', '더', '빙', '진짜', '짜증', '나', '네요', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍', '지', '않', '구나'],
 ['너무', '재', '밓었다그래서보는것을추천한다'],
 ['교도소', '이야기', '구먼', '솔직히', '재미', '없', '다', '평점', '조정'],
 ['사이몬페그',
  '익살',
  '스런',
  '연기',
  '돋보였',
  '던',
  '영화',
  '스파이더맨',
  '에서',
  '늙',
  '어',
  '보이',
  '기',
  '만',
  '했',
  '던',
  '커스틴',
  '던스트',
  '너무나',
  '이뻐',
  '보였',
  '다'],
 ['막',
  '걸음마',
  '뗀',
  '세',
  '부터',
  '초등',
  '학교',
  '학년',
  '생',
  '인',
  '살용',
  '영화',
  'ㅋㅋㅋ',
  '별반',
  '개',
  '아까움'],
 ['원작', '긴장감', '을', '제대로', '살려', '내', '지', '못했', '다'],
 ['별',
  '반개',
  '아깝',
  '다',
  '욕',
  '나온다',
  '이응경',
  '길용우',
  '연기',
  '생활',
  '몇',
  '년',
  '인지',
  '정말',
  '발',
  '로',
  '해도',
  '그것',
  '보단',
  '낫',
  '겟',
  '다',
  '납치',
  '감금',
  '만',
  '반복',
  '반복',
  '드라마',
  '가족',
  '없',
  '다',
  '연기',
  '못',
  '하',
  '사람',
  '만',
  '모엿',
  '네'],
 ['액션', '없', '는데', '재미', '있', '몇', '안', '되', '영화'],
 ['왜',
  '케',
  '평점',
  '낮',
  '건데',
  '꽤',
  '볼',
  '만',
  '한데',
  '헐리우드',
  '식

In [38]:
test_tokens[:3]

[['굳', 'ㅋ'],
 ['GDNTOPCLASSINTHECLUB'],
 ['뭐', '야', '평점', '나쁘', '진', '않', '지만', '점', '짜리', '더더욱', '아니', '잖아']]

## 2) Word Vectorize

* word embedding : 문장들을 word vector 형태로 변환
    * 윗단에서 tokenizer output을 '문장' 형태라고 가정하고 코드 작업
    1. 문장을 토큰으로 쪼갠다
    2. 쪼개진 토큰을 가장 긴 문장에 맞춰 패딩한다
    3. 패딩이 마친 토큰들을 word vector로 변환하다
* oov, padding, truncating 확인

## 3) Train, Valid, Test set 생성

# 3. Convolutional Neural Networks Modeling

## 1) 기본 Model 생성

* Check points
* padding, initializer
* activation function
* Dropout
* Batch Normalization
* optimizer

## 2) Model 저장, Callback 설정

# 4. Final Results